In [67]:
import pandas as pd
from scipy.sparse import csr_matrix
from sklearn.neighbors import NearestNeighbors
from fuzzywuzzy import process

In [68]:
movies='movies.csv'
ratings='ratings.csv'

# Defining data and data type
df_movies=pd.read_csv(movies, usecols=['movieId', 'title'], dtype={'movieId':'int32','title':'str'}) 
df_ratings=pd.read_csv(ratings, usecols=['userId', 'movieId', 'rating'], dtype={'userId':'int32', 'movieId':'int32', 'rating':'float'})

In [69]:
# Create a pivot table from the movie ratings DataFrame
# Each row represents a movie, each column represents a user, and the values are ratings
# Missing ratings are filled with 0
movies_users=df_ratings.pivot(index='movieId', columns='userId',values='rating').fillna(0)
# Convert the pivot table to a sparse matrix
mat_movies_users=csr_matrix(movies_users.values)

In [70]:
# Create a k-nearest neighbors (KNN) model for recommendation
# Using cosine similarity as the distance metric
# 'brute' algorithm is used to find nearest neighbors without building a search tree
# n_neighbors is set to 20, meaning it will consider 20 nearest neighbors for recommendation
model_knn= NearestNeighbors(metric='cosine', algorithm='brute', n_neighbors=20)

In [71]:
model_knn.fit(mat_movies_users)

NearestNeighbors(algorithm='brute', metric='cosine', n_neighbors=20)

In [72]:
def recommender(movie_name, data, model, n_recommendations):
    model.fit(data) # Fit the model to the data

    # Find the index of the selected movie in the data
    idx=process.extractOne(movie_name, df_movies['title'])[2]
    print('Movie Selected: ', df_movies['title'][idx], 'Index: ', idx)
    print('Searching for recommendations...')

    # Find the nearest neighbors and distances to the selected movie
    distances, indices=model.kneighbors(data[idx], n_neighbors=n_recommendations)
    print(distances, indices)

    # Print recommended movie titles
    for i in indices:
        print(df_movies['title'][i].where(i!=idx))

# Func usage
recommender('iron man', mat_movies_users, model_knn, 20)

Movie Selected:  Iron Man (2008) Index:  6743
Searching for recommendations...
[[7.77156117e-16 4.09144753e-01 4.13091640e-01 4.18127003e-01
  4.35628075e-01 4.58178695e-01 4.59801664e-01 4.72782777e-01
  4.85689712e-01 4.86523926e-01 4.87255711e-01 4.87463304e-01
  4.88252691e-01 4.99511904e-01 5.01248665e-01 5.03191641e-01
  5.05781131e-01 5.05905732e-01 5.09120306e-01 5.10883660e-01]] [[6743 7197 7195 7354 6726 7137 7026 7571 3880 6388 7601 6755 7022 7338
  4421 6195 5885 3740 7378 7289]]
6743                                            NaN
7197                                  Garage (2007)
7195                        Merry Madagascar (2009)
7354                             A-Team, The (2010)
6726                         Superhero Movie (2008)
7137                         Thirst (Bakjwi) (2009)
7026                                 Scorpio (1973)
7571                                 Win Win (2011)
3880                  Look Who's Talking Now (1993)
6388    After the Wedding (Efter br